In [1]:
%set_env CUDA_VISIBLE_DEVICES=0

from gpytorch.models.pyro_deep_gp import AbstractPyroHiddenGPLayer, AbstractPyroDeepGP
from gpytorch.variational import PyroExactVariationalStrategy, PyroSamplingVariationalStrategy, CholeskyVariationalDistribution
from gpytorch.kernels import MaternKernel, ScaleKernel, RBFKernel
from gpytorch.means import ConstantMean
from gpytorch.distributions import MultivariateNormal
from gpytorch.likelihoods import GaussianLikelihood
import math

import torch
import numpy as np

from scipy.cluster.vq import kmeans2

from pyro.infer import SVI, TraceMeanField_ELBO, Trace_ELBO
from pyro import optim
import pyro

from torch.utils.data import TensorDataset, DataLoader


env: CUDA_VISIBLE_DEVICES=0


In [2]:
!nvidia-smi

Fri Jun 21 15:25:22 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.39       Driver Version: 418.39       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:02:00.0  On |                  N/A |
| 30%   46C    P5    37W / 250W |   1893MiB / 10988MiB |      4%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:03:00.0 Off |                  N/A |
| 29%   33C    P8    20W / 250W |     11MiB / 10989MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [3]:

import bayesian_benchmarks
from bayesian_benchmarks.data import get_regression_data



class ToyHiddenGPLayer(AbstractPyroHiddenGPLayer):
    def __init__(self, input_dims, output_dims, name="", inducing_points=50):
        if type(inducing_points) == int:
            inducing_points = torch.randn(output_dims, inducing_points, input_dims)

        variational_distribution = CholeskyVariationalDistribution(
            num_inducing_points=inducing_points.size(-2),
            batch_size=output_dims
        )

        variational_strategy = PyroSamplingVariationalStrategy(
            self,
            inducing_points,
            variational_distribution,
            learn_inducing_locations=True
        )

        super().__init__(variational_strategy, input_dims, output_dims, True, name)

        batch_shape = torch.Size([output_dims])

        self.mean_module = ConstantMean(batch_shape=batch_shape)
        self.covar_module = ScaleKernel(
            MaternKernel(nu=2.5, batch_shape=batch_shape, ard_num_dims=input_dims),
            batch_shape=batch_shape
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)


In [4]:

#TODO: Double inheritance
class ToyDeepGP(AbstractPyroDeepGP):
    def __init__(self, input_dims, output_dims, total_num_data, hidden_gp_layers, likelihood, name="", inducing_points=50):
        inducing_points = torch.randn(output_dims, inducing_points, input_dims)

        variational_distribution = CholeskyVariationalDistribution(
            num_inducing_points=inducing_points.size(-2),
            batch_size=output_dims
        )

        variational_strategy = PyroSamplingVariationalStrategy(
            self,
            inducing_points,
            variational_distribution,
            learn_inducing_locations=True
        )

        super().__init__(
            variational_strategy,
            input_dims,
            output_dims,
            total_num_data,
            hidden_gp_layers,
            likelihood,
            name
        )

        batch_shape = torch.Size([output_dims])

        self.mean_module = ConstantMean(batch_shape=batch_shape)
        self.covar_module = ScaleKernel(
            MaternKernel(nu=2.5, batch_shape=batch_shape, ard_num_dims=input_dims),
            batch_shape=batch_shape
        )


    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

In [5]:
dataset='wilson_elevators'
dataset = get_regression_data(dataset)
N_train = dataset.X_train.shape[0]
N_test = dataset.X_test.shape[0]
D_X = dataset.D + 1
train_x, train_y = torch.tensor(dataset.X_train).float().cuda(), torch.tensor(dataset.Y_train[:, 0]).float().cuda()
test_x, test_y = torch.tensor(dataset.X_test).float().cuda(), torch.tensor(dataset.Y_test[:, 0]).float().cuda()
print("N_train = %d   N_test = %d" % (N_train, N_test))

pyro.set_rng_seed(0)
torch.manual_seed(0)

hidden_layer_width = 2
num_inducing = 50
inducing_points = (train_x[torch.randperm(N_train)[0:num_inducing], :])
inducing_points = inducing_points.clone().data.cpu().numpy()
inducing_points = torch.tensor(kmeans2(train_x.data.cpu().numpy(), inducing_points, minit='matrix')[0])
inducing_points = inducing_points.unsqueeze(0).expand((hidden_layer_width,) + inducing_points.shape)
inducing_points = inducing_points.to(device=train_x.device, dtype=train_x.dtype)

print(train_x.device, test_x.device)

N_train = 14939   N_test = 1660
cuda:0 cuda:0


In [6]:
likelihood = GaussianLikelihood().cuda()

hidden_gp = ToyHiddenGPLayer(
    train_x.size(-1),
    hidden_layer_width,
    name="layer1",
    inducing_points=inducing_points
).to(device=train_x.device, dtype=train_x.dtype)
deep_gp = ToyDeepGP(hidden_layer_width, 1, train_x.size(-2), [hidden_gp], likelihood, name="output_layer",
                    inducing_points=num_inducing).to(device=train_x.device, dtype=train_x.dtype)

hidden_gp.variational_strategy.variational_distribution.variational_mean.data = \
    0.2 * torch.randn(hidden_gp.variational_strategy.variational_distribution.variational_mean.shape, device=train_x.device, dtype=train_x.dtype)
deep_gp.variational_strategy.variational_distribution.variational_mean.data = \
    0.2 * torch.randn(deep_gp.variational_strategy.variational_distribution.variational_mean.shape, device=train_x.device, dtype=train_x.dtype)

train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=80, shuffle=True)

optimizer = optim.Adam({"lr": 0.03, "betas": (0.90, 0.999)})

deep_gp.annealing = 0.1
hidden_gp.annealing = 0.1

USE_NF = True


# different settings for u/f sampling versus f sampling (u marginalized out)
deep_gp.EXACT = hidden_gp.EXACT = False
num_particles = 4 if deep_gp.EXACT else 32
annealing_epoch = 0 if deep_gp.EXACT else 100
use_nf_epoch = 0 if USE_NF else 999999
n_epochs = 300 if deep_gp.EXACT else 500

elbo = TraceMeanField_ELBO(num_particles=num_particles, vectorize_particles=True, max_plate_nesting=1)
svi = SVI(deep_gp.model, deep_gp.guide, optimizer, elbo)

def ll_rmse(x, y, num_samples=50):
    pred = deep_gp(x, num_samples=num_samples)[:, :, 0]
    log_prob = torch.distributions.Normal(pred, (-0.5 * deep_gp.log_beta).exp()).log_prob(y)
    log_prob = torch.logsumexp(log_prob - math.log(num_samples), dim=0).mean()
    rmse = (pred.mean(0) - y).pow(2.0).mean().sqrt().item()
    return log_prob, rmse

In [7]:
print("Beginning training in EXACT=%s mode with %d particles" % (deep_gp.EXACT, num_particles))

for epoch_i in range(n_epochs):
    epoch_loss = 0
    if epoch_i == annealing_epoch:
        deep_gp.annealing = 1.0
        hidden_gp.annealing = 1.0
        if epoch_i > 0:
            print("Turning off KL annealing...")
    
    if epoch_i == use_nf_epoch:
        hidden_gp.use_nf = True
        deep_gp.use_nf = True

    for minibatch_i, (x_batch, y_batch) in enumerate(train_loader):
#         print(list(hidden_gp.named_parameters()))
        loss = svi.step(x_batch, y_batch)
        epoch_loss = epoch_loss + loss / len(train_loader)
    if epoch_i % 5 == 0 or epoch_i == (n_epochs - 1):
        train_ll, train_rmse = ll_rmse(train_x, train_y)
        test_ll, test_rmse = ll_rmse(test_x, test_y)
        precision = pyro.param('log_beta').exp().item()
        frmt = "[epoch %03d] loss: %.4f  test_ll: %.3f  train_ll: %.3f  test_rmse: %.3f  train_rmse: %.3f  obs_prec: %.3f"
        print(frmt % (epoch_i, epoch_loss, test_ll, train_ll, test_rmse, train_rmse, precision))


Beginning training in EXACT=False mode with 32 particles
[epoch 000] loss: 3.2658  test_ll: -1.496  train_ll: -1.563  test_rmse: 0.966  train_rmse: 1.000  obs_prec: 2.023
[epoch 005] loss: 0.5313  test_ll: -0.470  train_ll: -0.458  test_rmse: 0.386  train_rmse: 0.381  obs_prec: 6.227
[epoch 010] loss: 0.4695  test_ll: -0.490  train_ll: -0.486  test_rmse: 0.396  train_rmse: 0.394  obs_prec: 6.835
[epoch 015] loss: 0.4558  test_ll: -0.414  train_ll: -0.393  test_rmse: 0.366  train_rmse: 0.358  obs_prec: 7.370
[epoch 020] loss: 0.4419  test_ll: -0.392  train_ll: -0.372  test_rmse: 0.357  train_rmse: 0.349  obs_prec: 7.219
[epoch 025] loss: 0.4326  test_ll: -0.388  train_ll: -0.365  test_rmse: 0.357  train_rmse: 0.349  obs_prec: 8.046
[epoch 030] loss: 0.4272  test_ll: -0.389  train_ll: -0.375  test_rmse: 0.357  train_rmse: 0.352  obs_prec: 7.543
[epoch 035] loss: 0.4223  test_ll: -0.398  train_ll: -0.379  test_rmse: 0.359  train_rmse: 0.351  obs_prec: 6.939
[epoch 040] loss: 0.4257  test_

/home/jake.gardner/git/pyro/pyro/infer/trace_elbo.py:138: UserWarning: Encountered NaN: loss
  warn_if_nan(loss, "loss")


RuntimeError: 
                                                                               Trace Shapes:        
                                                                                Param Sites:        
                                                                                    log_beta       1
                                                               output_layer.gp_layer$$$means      50
                                                            output_layer.gp_layer$$$raw_vars      50
                                                            output_layer.gp_layer$$$log_beta       1
                                output_layer.gp_layer$$$variational_strategy.inducing_points 1 50  2
      output_layer.gp_layer$$$variational_strategy.variational_distribution.variational_mean    1 50
output_layer.gp_layer$$$variational_strategy.variational_distribution.chol_variational_covar 1 50 50
                                    output_layer.gp_layer$$$likelihood.noise

In [ ]:
%debug

In [ ]:
torch.float64

In [ ]:
!kill 45500